In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Лабораторная работа №4. Использование нейронных сетей для генерации текста
**Выполнил:** Китайский А.С.  
**Проверил:** Мохов А.С.

In [ ]:
import os
import time
import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.model_selection import train_test_split


In [ ]:
RANDOM_STATE = 42

In [ ]:
tf.config.list_physical_devices('GPU')[0]

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

### 1. Загрузка выборки стихотворений Пушкина

In [ ]:
poet = 'pushkin'

path_to_file = f'{poet}.txt'
path_to_file = tf.keras.utils.get_file(
    path_to_file, 
    f'http://uit.mpei.ru/git/main/TDA/raw/branch/master/assets/poems/{path_to_file}'
)

In [ ]:
# путь к файлу
path_to_file

'/root/.keras/datasets/pushkin.txt'

In [ ]:
# загрузим текст из файла
with open(path_to_file, encoding = "utf-8") as f:
    text = f.read()

print(f'Длина текста: {len(text)} символов')

Длина текста: 586731 символов


In [ ]:
print('Первые 500 символов текста:', text[:500], sep='\n\n')

Первые 500 символов текста:

Так и мне узнать случилось,
Что за птица Купидон;
Сердце страстное пленилось;
Признаюсь – и я влюблен!
Пролетело счастья время,
Как, любви не зная бремя,
Я живал да попевал,
Как в театре и на балах,
На гуляньях иль в воксалах
Легким зефиром летал;
Как, смеясь во зло Амуру,
Я писал карикатуру
На любезный женской пол;
Но напрасно я смеялся,
Наконец и сам попался,
Сам, увы! с ума сошел.
Смехи, вольность – всё под лавку
Из Катонов я в отставку,
И теперь я – Селадон!
Миловидной жрицы Тальи
Видел прел


### 2. Познакомимся с данными. Проанализируем статистические характеристики исходных данных (среднюю длину стихотворения, среднюю длину строки).

In [ ]:
EOS_TOKEN = '</s>'

def mean_line_len(poem):
    lines = [len(line.strip()) for line in poem.split('\n') if len(line.strip()) > 0]
    return sum(lines)/len(lines)

def describe_poems(text, return_df = False):
    '''Функция разбирает файл на отдельные стрихотворения и расчитывает их хар-ки'''
    poems_list = [poem.strip() for poem in text.split(EOS_TOKEN) if len(poem.strip()) > 0]
    df = pd.DataFrame(data=poems_list, columns=['poem'])
    df['len'] = df.poem.map(len)                     # длина
    df['lines'] = df.poem.str.count('\n')            # количество строк
    df['mean_line_len'] = df.poem.map(mean_line_len) # средняя длина строки
    if return_df:
        return df
    return df.describe()

In [ ]:
poem_df = describe_poems(text, return_df = True)
poem_df

,poem,len,lines,mean_line_len
0,"Так и мне узнать случилось,\nЧто за птица Купи...",2536,109,23.114286
1,"Хочу воспеть, как дух нечистый Ада\nОседлан бы...",5543,170,33.372671
2,"Покаместь ночь еще не удалилась,\nПокаместь св...",4279,131,33.451613
3,"Ах, отчего мне дивная природа\nКорреджио искус...",4435,131,33.364341
4,Арист! и ты в толпе служителей Парнасса!\nТы х...,3893,106,38.642857
...,...,...,...,...
714,Чудный сон мне бог послал —\n\nС длинной белой...,860,38,22.833333
715,"О нет, мне жизнь не надоела,\nЯ жить люблю, я ...",196,7,23.625000
716,"""Твой и мой, – говорит Лафонтен —\nРасторгло у...",187,5,30.333333
717,Когда луны сияет лик двурогой\nИ луч ее во мра...,269,7,32.750000


In [ ]:
describe_poems(text)

,len,lines,mean_line_len
count,719.000000,719.000000,719.000000
mean,808.037552,29.464534,27.445404
std,1046.786862,39.244020,5.854564
min,74.000000,5.000000,8.250000
25%,280.500000,9.000000,24.125000
50%,453.000000,16.000000,25.758065
75%,852.000000,33.000000,31.522727
max,8946.000000,437.000000,48.923077


### 3. Подготовим выборку для обучения

In [ ]:
# разобьем данные на тренировочные, валидационные и тестовые
train_poems, test_poems = train_test_split(poem_df.poem.to_list(), test_size=0.1, random_state=RANDOM_STATE)
train_poems, val_poems = train_test_split(train_poems, test_size=0.1, random_state=RANDOM_STATE)

train_poems = f'\n\n{EOS_TOKEN}\n\n'.join(train_poems)
val_poems = f'\n\n{EOS_TOKEN}\n\n'.join(val_poems)
test_poems = f'\n\n{EOS_TOKEN}\n\n'.join(test_poems)

In [ ]:
# создадим словарь уникальных символов из текста
vocab = sorted(set(text)) + [EOS_TOKEN]

print(f'{len(vocab)} уникальных символа')
print('Словарь:', vocab, sep='\n')

143 уникальных символа
Словарь:
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '\xa0', '«', '»', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ô', 'û', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', '–', '—', '„', '…', '</s>']


Закодируем текст с помощью слоя StringLookup

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), 
    mask_token=None
)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,    # сопоставление индексов с элементами словаря
    mask_token=None
)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1).numpy().decode('utf-8')
    
def ids_from_text(text):
    return ids_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))

In [ ]:
# пример кодирования
ids = ids_from_text(train_poems[:30])
res_text = text_from_ids(ids)
print('Исходный текст:', train_poems[:30], sep = '\n')
print('Закодированный текст:', ids.numpy(), sep = '\n')
print('Декодированный текст:', res_text, sep = '\n')

Исходный текст:
Корабль испанский трехмачтовый
Закодированный текст:
[ 87 120 122 106 107 117 134   2 114 123 121 106 119 123 116 114 115   2
 124 122 111 127 118 106 129 124 120 108 133 115]
Декодированный текст:
Корабль испанский трехмачтовый


In [ ]:
# кодируем данные и преобразуем их в датасеты
train_ids = ids_from_text(train_poems)
val_ids = ids_from_text(val_poems)
test_ids = ids_from_text(test_poems)

train_ids_dataset = tf.data.Dataset.from_tensor_slices(train_ids)
val_ids_dataset = tf.data.Dataset.from_tensor_slices(val_ids)
test_ids_dataset = tf.data.Dataset.from_tensor_slices(test_ids)

Разобьем текст на последовательности длины seq_length

In [ ]:
seq_length = 100
examples_per_epoch = len(train_ids_dataset) // (seq_length + 1)

In [ ]:
# параметры: размер последовательности и дроп последней уменьшенной последовательности
train_sequences = train_ids_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_ids_dataset.batch(seq_length+1, drop_remainder=True)
test_sequences = test_ids_dataset.batch(seq_length+1, drop_remainder=True)

# одна последовательность из train
for seq in train_sequences.take(1):
  print(text_from_ids(seq))

Корабль испанский трехмачтовый,
Пристать в Голландию готовый:
На нем мерзавцев сотни три,
Две обезьян


Создадим сдвиг target относительно input на один символ

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
train_dataset = train_sequences.map(split_input_target)
val_dataset = val_sequences.map(split_input_target)
test_dataset = test_sequences.map(split_input_target)

In [ ]:
for input_example, target_example in train_dataset.take(1):
    print("Input:", text_from_ids(input_example), sep='\n')
    print('\n\n')
    print("Target:", text_from_ids(target_example), sep='\n')

Input:
Корабль испанский трехмачтовый,
Пристать в Голландию готовый:
На нем мерзавцев сотни три,
Две обезья



Target:
орабль испанский трехмачтовый,
Пристать в Голландию готовый:
На нем мерзавцев сотни три,
Две обезьян


Перемешаем датасеты и разобьем их на батчи для оптимизации обучения

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

def prepare_dataset(dataset):
    dataset = (
        dataset
        .shuffle(BUFFER_SIZE) # перемешивание данных
        .batch(BATCH_SIZE, drop_remainder=True) # разбиение на батчи размером BATCH_SIZE
        .prefetch(tf.data.experimental.AUTOTUNE)) # загрузка данных в память заранее
    return dataset 

train_dataset = prepare_dataset(train_dataset)
val_dataset = prepare_dataset(val_dataset)
test_dataset = prepare_dataset(test_dataset)

train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### 4. Построим нейронную сеть. Тип ячейки GRU

Модель состоит из трех слоев

- tf.keras.layers.Embedding: Входной слой. Кодирует каждый идентификатор символа в вектор размерностью embedding_dim;
- tf.keras.layers.GRU: Рекуррентный слой на ячейках GRU в количестве gru_units
- tf.keras.layers.Dense: Выходной полносвязный слой размерностью vocab_size, в который выводится вероятность каждого символа в словаре.

In [ ]:
# длина словаря символов
vocab_size = len(vocab)

# размерность Embedding'а
embedding_dim = 256 #@param{type:"number"}

# Параметры GRU-слоя
gru_units = 300 #@param {type:"number"}
dropout_p = 0.5

T = 0.3 #@param {type:"slider", min:0, max:2, step:0.1}
N = 1000

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, gru_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(
        gru_units,             # размерность выхода
        dropout = dropout_p,
        return_sequences=True, # возвращает всю последовательность
        return_state=True)     # возвращает последнее состояние
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    
    states = self.gru.get_initial_state(x)

    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    gru_units=gru_units)

Иллюстрация работы сети:
![image](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Проверка необученой модели

In [ ]:
# посмотрим на один батч из датасета
for input_example_batch, target_example_batch in train_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print("(размер батча, длина последовательности, размер словаря) -", example_batch_predictions.shape)

(размер батча, длина последовательности, размер словаря) - (64, 100, 144)


In [ ]:
# посмотрим на логиты вероятности каждого символа на первой позиции
example_batch_predictions[0][0]

<tf.Tensor: shape=(144,), dtype=float32, numpy=
array([ 0.0027603 , -0.00812737,  0.00038508,  0.00031491,  0.01299553,
       -0.00303674,  0.00711669, -0.01294534,  0.01583236, -0.00627203,
       -0.00348256,  0.02125814, -0.00914515,  0.01366937, -0.00350473,
       -0.00257265,  0.00183849,  0.01162545,  0.00269023,  0.0090152 ,
       -0.01085356,  0.00805729,  0.00256377,  0.01179903, -0.00966423,
        0.00375698, -0.00679599,  0.0154108 ,  0.00180047,  0.01141567,
        0.0084399 ,  0.02246128, -0.02589275,  0.00823148, -0.01511441,
        0.00904186, -0.00039597, -0.00019091,  0.00401311, -0.0088466 ,
        0.00698363,  0.00074551, -0.00780546,  0.00096848,  0.0072231 ,
        0.00835975, -0.00634251, -0.01854552,  0.01088605,  0.00218695,
       -0.00554847, -0.00145263,  0.00696061,  0.00304085, -0.01234441,
       -0.01479187,  0.02132496, -0.00581373,  0.00124562, -0.00357364,
       -0.00045178, -0.00542335,  0.00045016, -0.00489227, -0.01009251,
       -0.007370

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([111,  95,  46,  38, 105, 114,  23,  70, 127,  66,  45,  75,   2,
        58,  85,  79,   9,  92,  51,  71, 126,  19, 125,  57,  58, 107,
       130,  50,  78,  79,   1,  89, 142,  78,  64,  19,  94, 135,  93,
        89,   9, 109,  56,   5,  54,  79,  29,  20, 103, 112, 128,  73,
        14, 136,  81, 132,  92, 100,  98,  69,  86,  26, 121,  24,  40,
       129, 129,  26,  95, 123,  74,  74,  91, 100,  68,  68,  19, 126,
        66, 129,  43, 112, 134,  90, 141, 132,  32,  59,  54,  95,  23,
        29, 114,  48,  86, 125,  81,  93,  83,  36])

In [ ]:
print("Input:", text_from_ids(input_example_batch[0]), sep='\n')
print()
print("Predictions:", text_from_ids(sampled_indices), sep='\n')

Input:
исходит светлый Мир.
Свершилось!.. Русской царь, достиг ты славной цели!
Вотще надменные на родину л

Predictions:
еТfWЯиFâх eô rИВ,ПkçфBуqrбшjБВ
М…БyBСэРМ,гp'nВNCЭжцé;юДъПЧХàЙJпH_ччJТсêêОЧ»»Bф чcжьН„ъQsnТFNиhЙуДРЖU


Настройка модели для обучения

In [ ]:
# функция потерь
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# пример
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: (размер батча, длина последовательности, размер словаря) -", example_batch_predictions.shape)
print("Mean loss:       ", example_batch_mean_loss)

Prediction shape: (размер батча, длина последовательности, размер словаря) - (64, 100, 144)
Mean loss:        tf.Tensor(4.9706573, shape=(), dtype=float32)


In [ ]:
print('perplexity: ',np.exp(example_batch_mean_loss))

perplexity:  144.1216


Перплексия приблизительно равна размеру словаря, что говорит о полной неопределенности модели при генерации текста.

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
model.summary()

Model: "my_model_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_40 (Embedding)    multiple                  36864     
                                                                 
 gru_27 (GRU)                multiple                  502200    
                                                                 
 dense_40 (Dense)            multiple                  43344     
                                                                 
Total params: 582,408
Trainable params: 582,408
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# используем tf.keras.callbacks.ModelCheckpoint, чтобы убедиться, 
# что контрольные точки сохраняются во время обучения:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor="val_loss",
    save_weights_only=True,
    save_best_only=True)

In [ ]:
# создадим модель, реализующую один шаг предсказания
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Создаем маску, чтобы предотвратить создание "[UNK]"
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # ставим знак "-" перед каждым неверным индексом
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    
  # Этот фрагмент целиком написан с использованием Tensorflow, поэтому его можно выполнять 
  # не с помощью интерпретатора языка Python, а через граф операций. Это будет значительно быстрее.  
  # Для этого воспользуемся декоратором  @tf.function   
  @tf.function   
  def generate_one_step(self, inputs, states=None, temperature=1.0):
    # преобразуем строки в идентификаторы токенов.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # запускаем модель
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # используем только последнее предсказание.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/temperature
    # применяем маску прогнозирования
    predicted_logits = predicted_logits + self.prediction_mask

    # используем выходные логиты для генерации идентификаторов токенов.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # преобразование идентификаторов токенов в символы
    predicted_chars = self.chars_from_ids(predicted_ids)


    # возвращаем символы и состояние модели
    return predicted_chars, states

### 5. Обучим нейронную сеть на разных количествах эпох (5, 15, 30, 50, 70) при зафиксированных параметрах embedding_dim = 256, gru_units = 300, T = 0.3 и сравним результаты генерации (тексты), перплексию и статистические характеристики сгенерированных текстов. Выберем оптимальное количество эпох

In [ ]:
NUM_EPOCHS = [5, 15, 30, 50, 70]

In [ ]:
df_epochs= pd.DataFrame(
    columns=['eval_loss', 'perplexity', 'result_text', 'run_time'], 
    index=['Epochs 5', 'Epochs 15', 'Epochs 30', 'Epochs 50', 'Epochs 70'])
dict_epochs_describe = {}

In [ ]:
for i in NUM_EPOCHS:
  model = MyModel(
        vocab_size=len(ids_from_chars.get_vocabulary()),
        embedding_dim=embedding_dim,
        gru_units=gru_units)
  model.compile(optimizer='adam', loss=loss)

  history = model.fit(
      train_dataset, 
      validation_data=val_dataset, 
      epochs=i, 
      callbacks=[checkpoint_callback]
  )
  eval_loss = model.evaluate(test_dataset)
  perplexity = np.exp(eval_loss)
  print('eval_loss:', eval_loss)
  print('perplexity', perplexity)
  
  one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

  start = time.time()
  states = None
  next_char = tf.constant(['\n'])
  result = [next_char]

  for n in range(N):
    next_char, states = one_step_model.generate_one_step(next_char, states=states, temperature=T)
    result.append(next_char)

  result = tf.strings.join(result)
  end = time.time()

  result_text = result[0].numpy().decode('utf-8')
  print(result_text)
  print('_'*80)
  run_time = end - start
  print('\nrun time:', run_time)

  df_epochs.loc[f'Epochs {i}'] = [eval_loss, perplexity, result_text, run_time]
  dict_epochs_describe[f'Epochs {i}'] = describe_poems(result_text)

Epoch 1/5
72/72 [==============================] - 25s 70ms/step - loss: 3.4753 - val_loss: 2.8282
Epoch 2/5
72/72 [==============================] - 3s 31ms/step - loss: 2.7048 - val_loss: 2.5678
Epoch 3/5
72/72 [==============================] - 3s 28ms/step - loss: 2.5607 - val_loss: 2.4815
Epoch 4/5
72/72 [==============================] - 2s 21ms/step - loss: 2.4768 - val_loss: 2.4026
Epoch 5/5
9/9 [==============================] - 0s 6ms/step - loss: 2.3432
eval_loss: 2.343165397644043
perplexity 10.414149368066521

Вой го мо пой пой ми ой,
Во пой насё пой,
Ви пой мей раза вы вой пой пой,
Ты,
Пой,
Пой гой ой бой сто помой бы ской стожестыхой пой ста помой пой – за ной вой бой пой ско ны ть ми вомой вой ны пой стара вы по стой ви пой дой дой пой вена, похой пой за стой да пой пой.
На зай ой ста сё пой вой на пелой вой,
И во пой вой дой пой пой вой пой пой пой зато вый мита вы, помемой зой вы ви пой и пра бой, мужи мой та да пой поми пой чу рыхой бе вой сты сть помий да той пой и 

In [ ]:
df_epochs

,eval_loss,perplexity,result_text,run_time
Epochs 5,2.343165,10.414149,"\nВой го мо пой пой ми ой,\nВо пой насё пой,\n...",2.991487
Epochs 15,1.960135,7.100286,"\nВо на сты ной, сты веве но вой сты веть сти ...",3.549093
Epochs 30,1.815334,6.143127,"\nКа у стой,\nВой веледни сто строй стой строй...",2.80781
Epochs 50,1.774156,5.895305,\nНодена стоде ста стой т велой вела стой пой ...,2.819697
Epochs 70,1.770683,5.874866,"\nНой от ве пой стодено ми стоде стоми м, стод...",2.842256


In [ ]:
for key, value in dict_epochs_describe.items():
  print(key, value, sep='\n')

Epochs 5
         len  lines  mean_line_len
count    1.0    1.0       1.000000
mean   999.0   14.0      65.666667
std      NaN    NaN            NaN
min    999.0   14.0      65.666667
25%    999.0   14.0      65.666667
50%    999.0   14.0      65.666667
75%    999.0   14.0      65.666667
max    999.0   14.0      65.666667
Epochs 15
          len  lines  mean_line_len
count     1.0    1.0       1.000000
mean   1000.0    2.0     332.666667
std       NaN    NaN            NaN
min    1000.0    2.0     332.666667
25%    1000.0    2.0     332.666667
50%    1000.0    2.0     332.666667
75%    1000.0    2.0     332.666667
max    1000.0    2.0     332.666667
Epochs 30
          len  lines  mean_line_len
count     1.0    1.0       1.000000
mean   1000.0    5.0     165.833333
std       NaN    NaN            NaN
min    1000.0    5.0     165.833333
25%    1000.0    5.0     165.833333
50%    1000.0    5.0     165.833333
75%    1000.0    5.0     165.833333
max    1000.0    5.0     165.833333
Epochs 5

По статистическим характеристикам сложно сделать какой-либо вывод, так как при любом количестве эпох модель находит только одно стихотворение. С увеличением количества эпох перплеския и ошибка на тестовых данных уменьшается, что говорит о том, что оптимальное количество эпох - 70.

### 6. Изменяя параметр температуры T проанализируем изменения сгенерированного текста. Выберем оптимальное значение параметра.

In [ ]:
T = [0.1, 0.3, 0.5, 0.7, 1.0]

In [ ]:
for i in T:
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states, temperature=i)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print(f"T = {i}", result_text, sep='\n')
    print('_'*80)
    print('\nRun time:', end - start)
    print('\n')

T = 0.1

Ве стой стой но стой стой стой ст стой сте стой стой стой стой стой стой стой ст сто стой сто ст ст стой сто ст стой сто сто сто сто ст стой ст стой ст стой стой ст ст ст ст стой сто стой стой стой стой сто стой стой стой ст стой стой стой стой стой стой сто ст сто стой сто сто сто сто стой стой сто стой пой стой ст сто стой ст стой ст ст стой стой стой стой стой стой ст стой стой стой стой ст ст сто стой сто ст стой ст сто стой ст стом сто ст ст стой ст сто ве стой ста стоде сто стой стой стой стой сто стой стой стой стой стой стой по стой сто сто стой сто стой стой стой сто сто стой сто стой стой стой стой стой сто стой сто сто сто стой стой стой ве стой стой сто сто стой стой стой стой стой стой стой сто сто стой сто стой стой сто стоде стой стой сто стой сто стой стой стой стой стой стой сто стой стой стой ст ве стой ст стой стой стой пой сто сто стой сто сто стостом стой стой сто ст сто стой стой стоде стой сто ст стой стой стой стой стой сто стой стой стой ст стой стой в

Визуально при T=0.7 текст имеет более разумный вид

### 7. Проанализируем зависимость перплексии, скорости обучения, результатов генерации от параметров нейронной сети embedding_dim, gru_units: embedding_dim = {vocab/4, vocab/2, vocab, vocab * 2, vocab * 4}, где vocab = размер словаря выборки. gru_units = {10, 100, 300, 500}

In [ ]:
embedding_dim = [vocab_size/4, vocab_size/2, vocab_size, vocab_size * 2, vocab_size * 4]
embedding_dim = list(map(int, embedding_dim))
gru_units = 300
T = 0.7
NUM_EPOCHS = 70

In [ ]:
df_embedding_dim = pd.DataFrame(
    columns=['eval_loss', 'perplexity', 'result_text', 'run_time'], 
    index=['embedding_dim ' + str(i) for i in embedding_dim])
dict_embedding_dim_describe = {}

In [ ]:
for i in embedding_dim:
    model = MyModel(
        vocab_size=len(ids_from_chars.get_vocabulary()),
        embedding_dim=i,
        gru_units=gru_units)
    model.compile(optimizer='adam', loss=loss)

    history = model.fit(
        train_dataset, 
        validation_data = val_dataset, 
        epochs=NUM_EPOCHS, 
        callbacks=[checkpoint_callback])
    
    eval_loss = model.evaluate(test_dataset)
    perplexity = np.exp(eval_loss)
    print('eval_loss:', eval_loss)
    print('perplexity', perplexity)

    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states, temperature=T)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print(result_text)
    print('_'*80)
    run_time = end - start
    print('\nRun time:', run_time)

    df_embedding_dim.loc[f'embedding_dim {i}'] = [eval_loss, perplexity, result_text, run_time]
    dict_embedding_dim_describe[f'embedding_dim {i}'] = describe_poems(result_text)

Epoch 1/70
72/72 [==============================] - 10s 82ms/step - loss: 3.7357 - val_loss: 3.3922
Epoch 2/70
72/72 [==============================] - 4s 37ms/step - loss: 3.1446 - val_loss: 2.8380
Epoch 3/70
72/72 [==============================] - 5s 36ms/step - loss: 2.8102 - val_loss: 2.6610
Epoch 4/70
72/72 [==============================] - 5s 36ms/step - loss: 2.6909 - val_loss: 2.5857
Epoch 5/70
72/72 [==============================] - 5s 37ms/step - loss: 2.6257 - val_loss: 2.5366
Epoch 6/70
72/72 [==============================] - 3s 17ms/step - loss: 2.5776 - val_loss: 2.4961
Epoch 7/70
72/72 [==============================] - 3s 17ms/step - loss: 2.5373 - val_loss: 2.4603
Epoch 8/70
72/72 [==============================] - 3s 16ms/step - loss: 2.5027 - val_loss: 2.4225
Epoch 9/70
72/72 [==============================] - 2s 13ms/step - loss: 2.4732 - val_loss: 2.3922
Epoch 10/70
72/72 [==============================] - 3s 23ms/step - loss: 2.4454 - val_loss: 2.3657
Epoch 11

In [ ]:
df_embedding_dim

,eval_loss,perplexity,result_text,run_time
embedding_dim 35,1.842625,6.313088,\nОгезродвопо нь стакой ва коме па бе звсть ни...,2.883097
embedding_dim 71,1.802408,6.064232,"\nОматse\nИлет яя дети,\nОзавый —\nГивиск – м;...",2.835564
embedding_dim 143,1.777167,5.913079,"\nНостени сихоре,\n\nСсты сточеголой по гой в ...",3.131208
embedding_dim 286,1.778821,5.922871,"\nИ па ме скра мытратый,\nНой ты езала мо све ...",3.408812
embedding_dim 572,1.784352,5.955718,"\nВу м\nПря м пленедем угде Токумегора,\nНу на...",2.831484


In [ ]:
for key, value in dict_embedding_dim_describe.items():
  print(key, value, sep='\n')

embedding_dim 35
              len      lines  mean_line_len
count    2.000000   2.000000       2.000000
mean   497.000000  14.500000      33.077778
std    275.771645   7.778175       0.738534
min    302.000000   9.000000      32.555556
25%    399.500000  11.750000      32.816667
50%    497.000000  14.500000      33.077778
75%    594.500000  17.250000      33.338889
max    692.000000  20.000000      33.600000
embedding_dim 71
          len  lines  mean_line_len
count     1.0    1.0           1.00
mean   1000.0   24.0          39.04
std       NaN    NaN            NaN
min    1000.0   24.0          39.04
25%    1000.0   24.0          39.04
50%    1000.0   24.0          39.04
75%    1000.0   24.0          39.04
max    1000.0   24.0          39.04
embedding_dim 143
          len  lines  mean_line_len
count     1.0    1.0       1.000000
mean   1000.0   33.0      31.193548
std       NaN    NaN            NaN
min    1000.0   33.0      31.193548
25%    1000.0   33.0      31.193548
50%    1000.

При увеличении размеров эмбеддингов перплексия и ошибка на тестовых данных уменьшались, улучшалось качество текстов, однако при размерности эмбеддинга 572 результаты стали хуже, что скорее всего свидетельствует о переобучении. Оптимальное значение параметра embedding dim - 286.

In [ ]:
embedding_dim = 256
gru_units = [10, 100, 300, 500]
T = 0.7
NUM_EPOCHS=70

In [ ]:
df_gru_units = pd.DataFrame(
    columns=['eval_loss', 'perplexity', 'result_text', 'run_time'], 
    index=['gru_units 10', 'gru_units 100', 'gru_units 300', 'gru_units 500'])
dict_gru_units_describe = {}

In [ ]:
for i in gru_units:
    model = MyModel(
        vocab_size=len(ids_from_chars.get_vocabulary()),
        embedding_dim=embedding_dim,
        gru_units=i)
    model.compile(optimizer='adam', loss=loss)

    history = model.fit(
        train_dataset, 
        validation_data = val_dataset, 
        epochs=NUM_EPOCHS, 
        callbacks=[checkpoint_callback])
    
    eval_loss = model.evaluate(test_dataset)
    perplexity = np.exp(eval_loss)
    print('eval_loss:', eval_loss)
    print('perplexity', perplexity)

    one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

    start = time.time()
    states = None
    next_char = tf.constant(['\n'])
    result = [next_char]

    for n in range(N):
        next_char, states = one_step_model.generate_one_step(next_char, states=states, temperature=T)
        result.append(next_char)

    result = tf.strings.join(result)
    end = time.time()

    result_text = result[0].numpy().decode('utf-8')
    print(result_text)
    print('_'*80)
    run_time = end - start
    print('\nRun time:', run_time)

    df_gru_units.loc[f'gru_units {i}'] = [eval_loss, perplexity, result_text, run_time]
    dict_gru_units_describe[f'gru_units {i}'] = describe_poems(result_text)

Epoch 1/70
72/72 [==============================] - 8s 79ms/step - loss: 4.5247 - val_loss: 3.9357
Epoch 2/70
72/72 [==============================] - 2s 21ms/step - loss: 3.7057 - val_loss: 3.5445
Epoch 3/70
72/72 [==============================] - 2s 15ms/step - loss: 3.4827 - val_loss: 3.4136
Epoch 4/70
72/72 [==============================] - 2s 11ms/step - loss: 3.3963 - val_loss: 3.3537
Epoch 5/70
72/72 [==============================] - 2s 10ms/step - loss: 3.3425 - val_loss: 3.2995
Epoch 6/70
72/72 [==============================] - 2s 11ms/step - loss: 3.2827 - val_loss: 3.2182
Epoch 7/70
72/72 [==============================] - 1s 8ms/step - loss: 3.1777 - val_loss: 3.1129
Epoch 8/70
72/72 [==============================] - 2s 11ms/step - loss: 3.0963 - val_loss: 3.0544
Epoch 9/70
72/72 [==============================] - 2s 11ms/step - loss: 3.0431 - val_loss: 3.0050
Epoch 10/70
72/72 [==============================] - 1s 8ms/step - loss: 2.9996 - val_loss: 2.9618
Epoch 11/70

In [ ]:
df_gru_units

,eval_loss,perplexity,result_text,run_time
gru_units 10,2.472713,11.85457,"\nРа ра, бу ут,\nКой т.\n\nНацака ис ся ны дья...",2.813268
gru_units 100,1.938313,6.947024,"\nTHiex помпося го\nБося когл!.\nБиходнь, сю д...",3.576499
gru_units 300,1.779971,5.929682,"\nДокамстлотов плета.\nИчедедавомнымы, разво и...",2.76825
gru_units 500,1.963069,7.121151,\nУголасететвь спломежненей новскру дитовецаюб...,2.974915


In [ ]:
for key, value in dict_gru_units_describe.items():
  print(key, value, sep='\n')

gru_units 10
          len  lines  mean_line_len
count     1.0    1.0       1.000000
mean   1000.0   26.0      37.461538
std       NaN    NaN            NaN
min    1000.0   26.0      37.461538
25%    1000.0   26.0      37.461538
50%    1000.0   26.0      37.461538
75%    1000.0   26.0      37.461538
max    1000.0   26.0      37.461538
gru_units 100
          len  lines  mean_line_len
count     1.0    1.0       1.000000
mean   1000.0   38.0      29.121212
std       NaN    NaN            NaN
min    1000.0   38.0      29.121212
25%    1000.0   38.0      29.121212
50%    1000.0   38.0      29.121212
75%    1000.0   38.0      29.121212
max    1000.0   38.0      29.121212
gru_units 300
          len  lines  mean_line_len
count     1.0    1.0          1.000
mean   1000.0   25.0         40.625
std       NaN    NaN            NaN
min    1000.0   25.0         40.625
25%    1000.0   25.0         40.625
50%    1000.0   25.0         40.625
75%    1000.0   25.0         40.625
max    1000.0   25.0   

При увеличении числа нейронов перплексия и ошибка на тестовы данных уменьшались, улучшалось качество текстов, время обучения увеличивалось, однако при 500 нейронов результаты стали хуже, что скорее всего свидетельствует о переобучении.